In [18]:
from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne
from pprint import pprint

# Establish client connection
client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']
courses = db.courses


# Bulk insert of courses with student enrollments
operations = [
    InsertOne({'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}),
    InsertOne({'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}),
    InsertOne({'course': 'History 201', 'enrollments': 20, 'department': 'History'}),
    InsertOne({'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'})
]
courses_collection.bulk_write(operations)
print('Courses inserted successfully.')
pprint(operations)

Courses inserted successfully.
[InsertOne({'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', '_id': ObjectId('69233b3a4c327f64d0858265')}),
 InsertOne({'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science', '_id': ObjectId('69233b3a4c327f64d0858266')}),
 InsertOne({'course': 'History 201', 'enrollments': 20, 'department': 'History', '_id': ObjectId('69233b3a4c327f64d0858267')}),
 InsertOne({'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics', '_id': ObjectId('69233b3a4c327f64d0858268')})]


In [19]:
query_high_enroll = {"enrollments": {"$gt": 20}}

for doc in courses.find(query_high_enroll):
    print(doc)

{'_id': ObjectId('69233b3a4c327f64d0858265'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('69233b3a4c327f64d0858266'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}


In [20]:
dept_filter = {"department": {"$in": ["Computer Science", "Mathematics"]}}

for doc in courses.find(dept_filter):
    print(doc)

{'_id': ObjectId('69233b3a4c327f64d0858265'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('69233b3a4c327f64d0858266'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}


In [21]:
avg_pipeline = [
    {"$group": {"_id": "$department", "avg_enrollment": {"$avg": "$enrollments"}}}
]

for record in courses.aggregate(avg_pipeline):
    print(record)

{'_id': 'History', 'avg_enrollment': 20.0}
{'_id': 'Computer Science', 'avg_enrollment': 25.0}
{'_id': 'Mathematics', 'avg_enrollment': 30.0}
{'_id': 'Physics', 'avg_enrollment': 15.0}


In [22]:
max_pipeline = [
    {"$group": {"_id": "$department", "max_enrollment": {"$max": "$enrollments"}}}
]

for record in courses.aggregate(max_pipeline):
    print(record)

{'_id': 'History', 'max_enrollment': 20}
{'_id': 'Computer Science', 'max_enrollment': 25}
{'_id': 'Mathematics', 'max_enrollment': 30}
{'_id': 'Physics', 'max_enrollment': 15}


In [23]:
projection_stage = [
    {
        "$project": {
            "course_name": "$course",
            "dept_name": "$department",
            "enrollments": 1,
            "_id": 0
        }
    }
]

for doc in courses.aggregate(projection_stage):
    print(doc)

{'enrollments': 30, 'course_name': 'Math 101', 'dept_name': 'Mathematics'}
{'enrollments': 25, 'course_name': 'CS 102', 'dept_name': 'Computer Science'}
{'enrollments': 20, 'course_name': 'History 201', 'dept_name': 'History'}
{'enrollments': 15, 'course_name': 'Physics 202', 'dept_name': 'Physics'}


In [24]:
category_stage = [
    {
        "$addFields": {
            "enrollment_category": {
                "$cond": [
                    {"$gt": ["$enrollments", 20]},
                    "high",
                    "low"
                ]
            }
        }
    }
]

for doc in courses.aggregate(category_stage):
    print(doc)

{'_id': ObjectId('69233b3a4c327f64d0858265'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'enrollment_category': 'high'}
{'_id': ObjectId('69233b3a4c327f64d0858266'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science', 'enrollment_category': 'high'}
{'_id': ObjectId('69233b3a4c327f64d0858267'), 'course': 'History 201', 'enrollments': 20, 'department': 'History', 'enrollment_category': 'low'}
{'_id': ObjectId('69233b3a4c327f64d0858268'), 'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics', 'enrollment_category': 'low'}


In [25]:
count_pipeline = [
    {"$group": {"_id": "$department", "total_courses": {"$sum": 1}}}
]

for doc in courses.aggregate(count_pipeline):
    print(doc)

{'_id': 'History', 'total_courses': 1}
{'_id': 'Computer Science', 'total_courses': 1}
{'_id': 'Mathematics', 'total_courses': 1}
{'_id': 'Physics', 'total_courses': 1}


In [27]:
filter_cs = [
    {"$match": {"department": "Computer Science", "enrollments": {"$gt": 25}}}
]

for doc in courses.aggregate(filter_cs):
    print(doc)

In [28]:
lookup_pipeline = [
    {
        "$lookup": {
            "from": "students",
            "localField": "course",
            "foreignField": "course_enrolled",
            "as": "student_info"
        }
    }
]

for doc in courses.aggregate(lookup_pipeline):
    print(doc)

{'_id': ObjectId('69233b3a4c327f64d0858265'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'student_info': []}
{'_id': ObjectId('69233b3a4c327f64d0858266'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science', 'student_info': []}
{'_id': ObjectId('69233b3a4c327f64d0858267'), 'course': 'History 201', 'enrollments': 20, 'department': 'History', 'student_info': []}
{'_id': ObjectId('69233b3a4c327f64d0858268'), 'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics', 'student_info': []}
